In [18]:
import pandas as pd 
import numpy as np
df = pd.read_csv('labelOne.csv')



In [19]:
print(df.info())
print("--------------")
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   numHullPoints  659 non-null    int64  
 1   numDefects     659 non-null    int64  
 2   bbox_width     659 non-null    int64  
 3   bbox_height    659 non-null    int64  
 4   aspect_ratio   659 non-null    float64
 5   area           659 non-null    float64
 6   perimeter      659 non-null    float64
 7   gesture_label  659 non-null    object 
dtypes: float64(3), int64(4), object(1)
memory usage: 41.3+ KB
None
--------------
       numHullPoints  numDefects  bbox_width  bbox_height  aspect_ratio  \
count     659.000000  659.000000  659.000000   659.000000    659.000000   
mean       19.127466   13.754173  144.781487   295.719272      0.546131   
std         2.768498    2.729490   49.738473    55.144947      0.366001   
min        11.000000    7.000000   85.000000   119.000000      0.244253  

In [20]:
# Remove rows with NaNs
df.dropna(inplace=True)

# Optional: Outlier removal
from scipy.stats import zscore
df = df[(np.abs(zscore(df.select_dtypes(include='number'))) < 3).all(axis=1)]
print(df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 624 entries, 20 to 658
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   numHullPoints  624 non-null    int64  
 1   numDefects     624 non-null    int64  
 2   bbox_width     624 non-null    int64  
 3   bbox_height    624 non-null    int64  
 4   aspect_ratio   624 non-null    float64
 5   area           624 non-null    float64
 6   perimeter      624 non-null    float64
 7   gesture_label  624 non-null    object 
dtypes: float64(3), int64(4), object(1)
memory usage: 43.9+ KB
None


In [21]:
print(df)

     numHullPoints  numDefects  bbox_width  bbox_height  aspect_ratio  \
20              17          13         125          306      0.408497   
28              15           9         122          307      0.397394   
29              18          11         120          315      0.380952   
30              15           9         123          323      0.380805   
31              15          10         124          330      0.375758   
..             ...         ...         ...          ...           ...   
654             22          16         213          225      0.946667   
655             11           7         192          245      0.783673   
656             15          10         184          257      0.715953   
657             15          10         181          262      0.690840   
658             16          12         180          265      0.679245   

        area  perimeter gesture_label  
20   14894.0    766.558      number_1  
28   14928.5    770.659      number_1  
29 

In [22]:
X = df.drop(columns=['gesture_label'])  # All numeric features
y = df['gesture_label']                 # Target labels


In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Use this as input to the model


In [24]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)  # Now 'number_1' becomes 0, etc.


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)


# Other training options
```python

Train a Lightweight Model (Good for Raspberry Pi)
Option A: SVM (Support Vector Machine)

Good accuracy, fast inference on small datasets

from sklearn.svm import SVC

model = SVC(kernel='linear')  # Try 'rbf' too if needed
model.fit(X_train, y_train)

Option B: Random Forest (if you want to understand feature importance)

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

Option C: Tiny Neural Network (for Edge Deployment)

from sklearn.neural_network import MLPClassifier

model = MLPClassifier(hidden_layer_sizes=(16,), max_iter=300)
model.fit(X_train, y_train)

```

In [26]:
# Training model 
from sklearn.svm import SVC

model = SVC(kernel='linear')  # Try 'rbf' too if needed
model.fit(X_train, y_train)


ValueError: The number of classes has to be greater than one; got 1 class

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=encoder.classes_))


In [ ]:
import joblib

joblib.dump(model, 'gesture_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(encoder, 'encoder.pkl')
